# Lunar Phase Calendar App
__[Github Repository](https://github.com/BobbyKostko/Astron1221-project2)__
## Project Motivation
- The moon's size and brightness in the sky often interfere with astronomical observation
- We wanted to create a tool designed for amateur astronomers who might not have the coding, data wrangling, or astronomy skills needed to accurately track the moon for good viewing conditions
- This puts a heavy emphasis on user-friendly implementation and accessibility
- We will generate data on lunar phase, rise/set times, and other important lunar events such as eclipses, packaging this data for easy use by others

## Coding Methods
- Pull data on earth/moon locations and rise/set times from the DE 421 ephemeris which is loaded through skyfield 
- Calculate illumination percentages and lunar events with this data, arranging our calculations in our own PANDAS table
- Store our data on GitHub and push it to a Streamlit app to generate lunar reports on nightly phase and other noteable lunar events
- Streamlit implementation means nobody needs to meet the installation dependencies of our code or know how to run it. They dont even need to run it at all since the data is nestled in our repository.

## Installations and Requirements
### As listed in our repository's requirements.txt file, to run the data generation code, you need to install:
- Pandas
- Skyfield
- Numpy
  
### Additionally, our web app has separate requirements (also in requirements.txt):
- Streamlit
- Plotly (for data visualizations)
- Pillow (for calendar generation)

## Laying the Foundations
We start with making all of our imports, loading the proper ephemeris, and establishing a few objects and planetary constants used throughout the data generation process.

In [ ]:
from datetime import datetime, timezone, timedelta
from skyfield.api import load, Topos
from skyfield import almanac
import numpy as np
import pandas as pd
from zoneinfo import ZoneInfo
import os

# Load timescale and ephemeris
ts = load.timescale()
eph = load('de421.bsp')

earth = eph['earth']
moon = eph['moon']
sun = eph['sun']

# Eclipse constants (angular sizes in degrees)
EARTH_ANGULAR_RADIUS_AT_MOON = 1.9   # Earth's angular radius as seen from Moon
SUN_ANGULAR_RADIUS_AT_MOON = 0.27   # Sun's angular radius as seen from Moon (~1AU)

## Calculating Lunar Phase
The first pieces of data we want to get are the moon's phase and illumination percentage at a given time. This is relatively easy using skyfield's almanac feature which already contains this information, but we need to make sure the value it outputs is in the format we want.
### Retrieving phase and illumination percentage from our ephemeris

In [ ]:
def get_lunar_phase(date):
    """
    Get the lunar phase for a given UTC datetime.
    
    Args:
        date: UTC-aware datetime object
        
    Returns:
        phase_name: String name of the phase (e.g., "Full Moon")
        illumination: Percentage of moon illuminated (0-100%)
    """
    t = ts.utc(date)
    
    # Calculate elongation (angle between sun and moon as seen from Earth)
    # using Skyfield's built-in moon_phase function
    phase_angle = almanac.moon_phase(eph, t)
    elongation = phase_angle.degrees
    
    # Calculate illumination percentage
    # elongation: 0° (New) -> 180° (Full) -> 360° (New)
    illumination = (1 - abs(elongation - 180) / 180) * 100
    illumination = round(illumination, 1)

Skyfield's almanac gives us the angle between the sun and moon as seen from earth, which corresponds to the percentage of the moon's illuminated side we can see.
### Classifying phase names for our retireved angles

In [ ]:
# Determine phase name
    if elongation < 22.5 or elongation >= 337.5:
        phase_name = "New Moon"
    elif elongation < 67.5:
        phase_name = "Waxing Crescent"
    elif elongation < 112.5:
        phase_name = "First Quarter"
    elif elongation < 157.5:
        phase_name = "Waxing Gibbous"
    elif elongation < 202.5:
        phase_name = "Full Moon"
    elif elongation < 247.5:
        phase_name = "Waning Gibbous"
    elif elongation < 292.5:
        phase_name = "Last Quarter"
    else:
        phase_name = "Waning Crescent"
    
    return phase_name, illumination

Now we connect our phase angles into something more easily understandable, phase names. It is worth noting here that a "full moon" by our definition only needs 90% of the moon's visible surface illuminated rather than 100%. We chose to do this for two reasons:
- For an amateur astronomer looking either at the moon or trying to guage how much the moon affects sky visibility on a given night, 90% is good enough to be almost indistinguishable from 100%.
- Our code will only calaulate lunar phase once each day for efficiency. With illumination percentages slightly changing over a 24-hour period, we will almost never calculate 100% illumination.

## Finding Rise/Set Times
The next pieces of data we want to retrieve are the moon's rise and set times each day. This is again fairly simple with Skyfield almanac since it already has a function for finding rise and set times of any object when viewed from any earth coordinates.
### Note: This is locally centered data
We are choosing our coordinates to be for Columbus specifically. Ultimately we will convert this data from UTC to EST in our final app.

In [ ]:
def get_moon_rise_set(date, latitude=39.9612, longitude=-82.9988, elevation_m=275.0):
    """
    Get moon rise and set times for a given calendar day using Skyfield almanac.
    
    Args:
        date: UTC-aware datetime object
        latitude: Observer's latitude in degrees (default: 39.9612, Columbus, OH)
        longitude: Observer's longitude in degrees (default: -82.9988, Columbus, OH)
        elevation_m: Observer's elevation in meters (default: 275.0, Columbus, OH)
    
    Returns:
        rise_time: datetime (UTC) of first moonrise in the day, or None
        set_time: datetime (UTC) of first moonset in the day, or None
    """
    # Observer location
    site_topos = Topos(latitude_degrees=latitude,
                       longitude_degrees=longitude,
                       elevation_m=elevation_m)

    # Start/end of the UTC day
    start_of_day = date.replace(hour=0, minute=0, second=0, microsecond=0)
    end_of_day = start_of_day + timedelta(days=1)

    t0 = ts.utc(start_of_day)
    t1 = ts.utc(end_of_day)

    # Build above-horizon function and find discrete transitions
    above_horizon_fn = almanac.risings_and_settings(eph, moon, site_topos)
    times, events = almanac.find_discrete(t0, t1, above_horizon_fn)

    # Extract first rise and set times within the day
    rise_time = None
    set_time = None
    for t, ev in zip(times, events):
        if ev == 1 and rise_time is None:  # rising edge
            rise_time = t.utc_datetime()
        elif ev == 0 and set_time is None:  # setting edge
            set_time = t.utc_datetime()

        if rise_time is not None and set_time is not None:
            break

    return rise_time, set_time

## Checking for Eclipses
The next event we want to check for is a lunar eclipse. This requires some slight geometry to determine if the moon appears within the earth's shadow at a given time and if so, by how much.

In [ ]:
def check_lunar_eclipse(date):
    """
    Check for lunar eclipse at given time.
    Returns: (eclipse_type, shadow_depth, elongation) where:
        eclipse_type: "Total", "Partial", "Penumbral", or None
        shadow_depth: 0-100 indicating shadow coverage
        elongation: angle from opposition
    """
    if isinstance(date, str):
        date = datetime.strptime(date, '%Y-%m-%d')
    if date.tzinfo is None:
        date = date.replace(tzinfo=timezone.utc)
    
    t = ts.utc(date)
    sun_vec = earth.at(t).observe(sun).apparent()
    moon_vec = earth.at(t).observe(moon).apparent()
    
    # Elongation check: must be near opposition (full moon)
    elongation = sun_vec.separation_from(moon_vec).degrees
    if abs(elongation - 180) > 5:  # Not close enough to opposition
        return None, 0, abs(elongation - 180)

We start by getting our sun and moon positions as seen from earth to find their elongation angle again.A lunar eclipse can only happen when the moon is roughly behind the earth (since that's where earth's shadow appears) so we only run the eclipse calculation for elongation angles close to 180 to save some computing time.

In [ ]:
# Shadow cone angles (in degrees) - based on angular sizes
    # Penumbra: Earth + Sun angular radii
    # Umbra: Earth - Sun angular radii  
    penumbra_radius = EARTH_ANGULAR_RADIUS_AT_MOON + SUN_ANGULAR_RADIUS_AT_MOON
    umbra_radius = EARTH_ANGULAR_RADIUS_AT_MOON - SUN_ANGULAR_RADIUS_AT_MOON
    
    # Offset from perfect opposition
    offset = abs(elongation - 180)
    
    # Classify eclipse
    if offset < umbra_radius * 0.5:
        return "Total", int(100 * (1 - offset / umbra_radius)), offset
    elif offset < umbra_radius:
        return "Partial", int(100 * (1 - offset / umbra_radius)), offset
    elif offset < penumbra_radius:
        return "Penumbral", int(50 * (1 - offset / penumbra_radius)), offset
    return None, 0, offset

Now we need to get our radii for earth's shadow using the angular radius constants defined at the start of our code. Since the center of earth's shadow appears at a perfect 180%, we only need to see if the elongation angle of the moon appears within the range of our shadow's radius from 180%. This will tell us if the moon's center appears within the umbral or penumbral part of earth's shadow at any given time.

## Samplling Each Night for an Eclipse
A lunar eclipse only lasts a few hours, so if we only check for a lunar eclipse once a night, we might miss every single one. Instead, we need to conduct regular checks throughout every night while the moon is up.

In [ ]:
def sample_night_for_eclipse(date_utc, rise_time, set_time):
    """
    Sample throughout a single night (hourly) to find maximum eclipse.
    Returns: (eclipse_type, shadow_depth, max_eclipse_time_utc)
    """
    if not rise_time or not set_time:
        return None, 0, None
    
    best_eclipse_type = None
    best_depth = 0
    best_time = None
    
    # Get the sampling window from rise/set times
    night_start = rise_time
    night_end = set_time
    # If set time is before rise time on the calendar, moon sets next day
    if night_end < night_start:
        night_end = night_end + timedelta(days=1)
    
    # Sample every hour throughout the visible period
    current_time = night_start
    max_samples = 48  # Safety: max 2 days of hourly samples
    sample_count = 0

We will conduct hourly checks for every rise/set window for a lunar eclipse. We are looking for a simple, easily digestible lunar report, so we are only keeping track of the eclipse's peak as the night goes on. We keep running variables on the peak eclipse depth, the kind of eclipse, and the time in which that peak occurs.

In [ ]:
while current_time <= night_end and sample_count < max_samples:
        eclipse_type, depth, _ = check_lunar_eclipse(current_time)
        
        # Track best eclipse found
        if eclipse_type and depth > best_depth:
            best_eclipse_type = eclipse_type
            best_depth = depth
            best_time = current_time
        
        current_time += timedelta(hours=1)
        sample_count += 1
        
        # Safety check: don't sample beyond 48 hours
        if (current_time - night_start).total_seconds() > 86400 * 2:
            break
    
    return best_eclipse_type, best_depth, best_time